__Code cell below defines utility functions for displaying math operations__

The utility functions are:

* `retrieve_name()`: Not for the user. Used by the other functions to find the name/symbol assigned to an expression. Only looks in the wrapping function name space. It does not search all the way up to the global name space.
* `dmo_auto()`: Calling this function causes dmo aware functions (so far only `diff(expr)` and `integ(expr)` defined below) to display a typeset version of the operations carried out by the function. The call `dmo_auto(status=False)` turns this off.
* `dmo()`: Passing a valid sympy expression or assignment statement to this function causes the expression or assignment to be evaluated and the operation output as typeset math. Also takes the optional argument `code=True`, causing it to try to output the result of the operation as plain code as well. This will just yield another typeset version of the result if `sympy.init_printing(pretty_print=True)` is set rather than `sympy.init_printing(pretty_print=False)`.

In [1]:
from IPython.display import HTML as html, HTML
from IPython.display import display
import sympy as sp
import inspect

def retrieve_name(var):# Used to identify the string used to name an expression
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    string_name = ''
    for var_name, var_val in callers_local_vars:
        if (var_val is var) and not (var_name.startswith('_')):
            string_name = var_name
    return string_name

def dmo_auto(status=True):
    '''
    sets the status of dmo (display math operation) to True by default or False if `status=False`
    is passed to the function. If true dmo aware functions will attempt to display typeset
    expressions for the operations carried out. If False they will only display typeset expressions
    for the operations if `display_op=True` is set in the function call. See the documentation for
    each function.
    '''
    globals()['_dmo_auto_']=status
    return('Automatic display of math operation activated for `dmo aware` operations.')

def dmo(*exprs,**kwarg): #Display math operation
    '''
    Pass one of:
    *exprs:     any valid symbolic math expression or function call (e.g. 2*x, sin(2*x)).
    **kwarg:    any valid symbol equal to an expression or function call (e.g. P=n*R*T/V,
                r=sin(theta)). This must be the first item with an equal sign appearing
                in the list of arguments.
                
                additionally you can add these options:
                code=True, if you want the code version of the evaluated expression to
                    appear in the cell output. Useful if you want to copy and edit into
                    another code cell. Note if `sympy.init_printing(pretty_print=True)` is
                    set the code will still be displayed as typeset. Call `sympy.init_printing`
                    with `pretty_print=False` to get non-typeset output.
    
    In an IPython/Jupyter notebook will display a typeset version of the operation and return
    the expression to the namespace for further operations. The function tries to identify
    the actual string (name) passed for the expression and output the typeset expression 
    `name=result of the operation`. If it cannot identify the actual string passed it just
    outputs the typeset result of the operation.
    
    J. Gutow May 2020.
    '''
    code = kwarg.pop('code', None)
    expr=None
    if(len(exprs) >= 1):
        expr = exprs[0] #ignore others.
        namestr=retrieve_name(expr)
        if (namestr==''):
            display(html(r'$'+sp.latex(expr)+'$'))
        else:
            display(html(r'$'+namestr+'='+sp.latex(expr)+'$'))
    else:
        key = list(kwarg)[0] #ignore all but first.
        expr = kwarg[key]
        display(html(r'$'+(key)+'='+sp.latex(expr)+'$'))
        globals()[key]=expr #inject into namespace.
    globals()['_']=expr #inject into last result
    if (code):
        return(expr)
    pass

__Code cell below defines dmo (display math operation) aware versions of sympy functions__

So far they are:

* `diff()`: Function that replaces the `sympy.diff()` function. Has all the same features as the sympy function as it just wraps it to provide the additional features of being dmo aware and to improve the versatility of the typeset output of partial differential symbols.
* `integ()`: Function that replaces the `sympy.integrate\Integral()` function(s). Has all the same features as the sympy functions as it just wraps them to provide the additional feature of being dmo aware.

In [2]:
def diff(f,*symbols,**kwargs):
    '''
    NB: SOME OF THE PARSING AND ORDERING OF DENOMINATORS IN DIFFERENTIALS IS NOT RIGHT YET
    Wrapper for sympy.diff(). That can output typeset expressions for the operations carried out before
    returning the result. Designed primarily for teaching and display purposes.
    
    This function is display math operation (dmo) aware. Thus if dmo_auto() is called at the beginning of
    a notebook all calls will result in typeset operation output unless the optional kwarg: display_op is
    set to False.
    
    Takes one additional optional kwarg that is not used by sympy.diff():
        display_op: True generates typeset math operation output, False suppresses the output.
    '''
    def symparse(symbols):
        syms=[]
        pwrs =[]
        for k in range(len(symbols)):
            tempsym = symbols[k]
            #print('tempsym = '+str(tempsym))
            if not (type(tempsym) is int):
                if(tempsym not in syms):
                    syms.append(tempsym)
                    pwrs.append(1)
                else:
                    for j in range(len(syms)):
                        if (tempsym==syms[j]):
                            pwrs[j]+=1
            else:
                for j in range(len(syms)):
                    if (symbols[k-1]==syms[j]):
                        pwrs[j]+=tempsym-1
        return(syms,pwrs)

    display_op=kwargs.pop('display_op',None) # For getting nice display from bare function.
    result=sp.diff(f,*symbols,**kwargs)
    try:
        dmo_auto = globals()['_dmo_auto_']
    except KeyError:
        dmo_auto=False
    if (display_op==False): #force overide of dmo_auto
        dmo_auto=False
    oper=None
    namestr=retrieve_name(f)
    if (dmo_auto) or (display_op):
        syms,pwrs = symparse(symbols)
        totpwr = sum(pwrs)
        totpwrstr=''
        if not(totpwr==1):
            totpwrstr=str(totpwr)
        ltop = ''
        ltdenom=''
        for i in range(len(syms)):
            pwrstr=''
            if not(pwrs[i]==1):
                pwrstr=str(pwrs[i])
            else:
                pwrstr=''
            ltdenom+='\\partial '+str(syms[i])+'^{'+pwrstr+'} '
        if (namestr==''):
            ltnum='\\partial^{'+totpwrstr+'} '
            ltop='\\frac{'+ltnum+'}{'+ltdenom+'}{\\left('+sp.latex(f)+'\\right)}'
        else:
            ltnum ='\\partial^{'+totpwrstr+'}'+str(namestr)+' '
            ltop ='\\frac{'+ltnum+'}{'+ltdenom+'}='
            ltnum='\\partial^{'+totpwrstr+'} '
            ltop+='\\frac{'+ltnum+'}{'+ltdenom+'}{\\left('+sp.latex(f)+'\\right)}'
        display(HTML('$'+ltop+'='+sp.latex(result)+'$'))
    return(result)

def integ(f, *args, **kwargs):
    '''
    Wrapper for `sympy.integrate()/Integral()`. That can output typeset expressions for the operations
    carried out before returning the result. Designed primarily for teaching and display purposes.
    
    This function is display math operation (dmo) aware. Thus if dmo_auto() is called at the beginning of
    a notebook all calls will result in typeset operation output unless the optional kwarg: `display_op` is
    set to False.
    
    Takes one additional optional kwarg that is not used by sympy.integrate():
        display_op: True generates typeset math operation output, False suppresses the output.

    '''
    display_op=kwargs.pop('display_op',None) # For getting nice display from bare function.
    result=sp.integrate(f,*args,**kwargs)
    try:
        dmo_auto = globals()['_dmo_auto_']
    except KeyError:
        dmo_auto=False
    if (display_op==False): #force overide of dmo_auto
        dmo_auto=False
    oper=None
    if (dmo_auto) or (display_op):
        namestr=retrieve_name(f)
        ltop1=''
        if not(namestr==''):
            oper1=sp.Integral(namestr,*args,**kwargs)
            ltop1=sp.latex(oper1)+'='
        oper=sp.Integral(f,*args,**kwargs)
        # TODO show subtraction step in computing integrals with limits
        #      question: best way to represent this for multiple integrals?
        display(HTML('$'+ltop1+sp.latex(oper)+'='+sp.latex(result)+'$'))
    return(result)

In [3]:
# Turning off sympy auto pretty printing as that seems to be the default in latest Jupyter/Sympy.
sp.init_printing(pretty_print=False)
# Turning on auto display math operations
dmo_auto()

'Automatic display of math operation activated for `dmo aware` operations.'

_Bare expressions_

In [4]:
sp.var('a b c')
b*c

b*c

In [5]:
dmo(b*c)

In [6]:
# Example with `code=True`.
dmo(b*c,code=True)

b*c

In [7]:
# As with normal cell execution `code=True` only shows result of last statement
dmo(sp.sin(b*c),code=True)
dmo(sp.cos(b*c),code=True)

cos(b*c)

_Demonstratons of `diff()` function compared with `sp.diff()`_

In [8]:
dmo(p = a*b/c**2)
print('`diff(p,c)`:')
diff(p,c)
print('This generates two outputs. `dmo(diff(p,c))`:')
dmo(diff(p,c))
print('`dmo(sp.diff(p,c))`:')
dmo(sp.diff(p,c))
print('`sp.diff(p,c)`:')
sp.diff(p,c)

`diff(p,c)`:


This generates two outputs. `dmo(diff(p,c))`:


`dmo(sp.diff(p,c))`:


`sp.diff(p,c)`:


-2*a*b/c**3

In [9]:
# higher order derivatives
print('`diff(p,c,b)`:')
diff(p,c,b)
print('This shows the steps. `diff(diff(p,c),b)`:')
diff(diff(p,c),b)
print('`diff(p,c,b,c)`:')
diff(p,c,b,c)
print('`diff(p,c,2)`:')
diff(p,c,2)

`diff(p,c,b)`:


This shows the steps. `diff(diff(p,c),b)`:


`diff(p,c,b,c)`:


`diff(p,c,2)`:


6*a*b/c**4

_Demonstration of `integ()` as compared to `sp.integrate()`_

In [10]:
print('`integ(p,c)`:')
integ(p,c)
print('This generates two outputs. `dmo(integ(p,c))`:')
dmo(integ(p,c))
print('`dmo(sp.integrate(p,c))`:')
dmo(sp.integrate(p,c))
print('`sp.integrate(p,c)`:')
sp.integrate(p,c)

`integ(p,c)`:


This generates two outputs. `dmo(integ(p,c))`:


`dmo(sp.integrate(p,c))`:


`sp.integrate(p,c)`:


-a*b/c

In [11]:
# with limits
dmo(c12=integ(p,(c,1,2)))
integ(p,(c,1,2))

a*b/2

_Assignment statements_

In [12]:
sp.var('a b c theta psi')
dmo(p=psi*b/c)

In [13]:
# Properly assigns the result to `_` so last result can be used with that symbol.
_

b*psi/c

In [14]:
# Example with `code=True`.
dmo(M=sp.Matrix([[1,2],[3,4]]),code=True)

Matrix([
[1, 2],
[3, 4]])

In [15]:
dmo(M)

In [16]:
dmo(T=M.transpose())

In [17]:
dmo(Meigvals=M.eigenvals())
dmo(Meigvecs=M.eigenvects())

In [18]:
dmo(q=sp.exp(-theta**2/b))

In [19]:
dmo(r=sp.sin(a*sp.pi/c))

In [20]:
dmo(sp.diff(r,c))
dmo(drdc = sp.diff(r,c))

In [21]:
dmo(d2rdcda=sp.diff(drdc,a))

In [22]:
dmo(d2rdcda=diff(drdc,a))

In [23]:
dmo(psi=sp.exp(-sp.I*b))

In [24]:
dmo(psi)

In [25]:
dmo(psi1=psi.evalf(4,subs={b:2.35}))

In [26]:
dmo(psi1)

In [27]:
_

-0.7027 - 0.7115*I

In [28]:
# abuse of pretty_print. It does not handle what dmo does.
from sympy import *
var('ppa ppb ppc')
pretty_print(ppd=ppa*ppb/ppc)

TypeError: pretty_print() got an unexpected keyword argument 'ppd'

In [2]:
import sympy as sp
sp.Min?

In [6]:
type(2)

int

In [8]:
type(2.0)

float